### This program calculates the spaial error and shape error between a reference image and a drawing
- Shape Error: Measured by MSE
- Spatial Error: Rotation, Translation, Scaling

#### Load dependencies and define directory hierarchy

In [1]:
import torch as th

import matplotlib.pyplot as plt
import time

import sys
import os
from __future__ import division
import os
import urllib, cStringIO
import pymongo as pm ## first establish ssh tunnel to server where database is running
import base64
import numpy as np
from numpy import *
import PIL
from PIL import Image
import base64
import matplotlib
%matplotlib inline
from matplotlib import pylab, mlab, pyplot
from IPython.core.pylabtools import figsize, getfigs
plt = pyplot
import seaborn as sns
sns.set_context('poster')
sns.set_style('white')
from matplotlib.path import Path
import matplotlib.patches as patches
import cv2
import pandas as pd
from svgpathtools import parse_path
import svg_distance_helpers as rsh
import torch
from torch.autograd import Variable
import SimpleITK as sitk
#sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(__file__))))
import airlab as al
from torch.autograd import Variable
import affine_registration_2d as ar
from scipy.stats import pearsonr

### File Hierarchy

In [2]:
# directory & file hierarchy
analysis_dir = os.getcwd()
ref_dir = os.path.join(analysis_dir, 'tracing_ref_400')

loss = 'ncc'
trans_dir = os.path.join(analysis_dir, 'transformed_{}'.format(loss))

canvas_side = 400 # image size
rows, cols = canvas_side, canvas_side # the size of a given drawing image
#tracing = ['this square','this shape', 'this circle']
tracing = ['square','shape', 'this circle']
data_output = 'kiddraw_tracing_{}.csv'.format(loss)

In [3]:
# set vars 
auth = pd.read_csv('auth.txt', header = None) # this auth.txt file contains the password for the sketchloop user
pswd = auth.values[0][0]
user = 'sketchloop'
host = 'rxdhawkins.me' ## cocolab ip address

# have to fix this to be able to analyze from local
import pymongo as pm
conn = pm.MongoClient('mongodb://sketchloop:' + pswd + '@127.0.0.1')
response_db = conn['kiddraw']
coll = response_db['tracing_eval']

stimuli_db = conn['stimuli']
scoll = stimuli_db['kiddraw_tracing_eval']

# which iteration name should we use?
iterationName = 'pilot0'

### Basic Information from Current Data

In [4]:
print "There are {} ratings in total".format(coll.find({'iterationName':iterationName, 'workerId':{'$exists':True}}).count())

rated_img_ids = coll.distinct('session_id')
print "{} unique tracing images get rated by one or more mturkers".format(len(rated_img_ids))

p_id = []
m_id = []
for s in rated_img_ids:
    if s.startswith('cdm_run'):
        m_id.append(s)
    elif s.startswith('CDM_photodraw'):
        p_id.append(s)

print "{} tracing images from museumstation are rated".format(len(m_id))
print "{} tracing images from photodraw are rated".format(len(p_id))

There are 2479 ratings in total
1083 unique tracing images get rated by one or more mturkers
1021 tracing images from museumstation are rated
60 tracing images from photodraw are rated


In [16]:
# get data from csv file
data = pd.read_csv(data_output)
try:
    data = data.drop("Unnamed: 0", axis=1)
except:
    print "Already droped"

In [17]:
print data.count()

age                  2154
human_norm_rating    1151
human_rating         1151
norm_r               1151
norm_s               1151
norm_t               1151
post_tran            2154
pre_tran             2154
rotate               2154
scale                2154
session_id           2154
spatial              1151
tracing_item         2154
translate            2154
dtype: int64


In [7]:
# generate reference shapes
really_run_this = 1
if really_run_this:
    ref_path = []
    for t in tracing:
        print t
        fname = os.path.join(ref_dir, '{}.png'.format(t))
        img_ref = cv2.imread(fname)
        img_ref = rsh.color_to_num(img_ref, threshold=0, white_background=False)
        ofname = os.path.join(ref_dir, '{}_ref.png'.format(t))
        cv2.imwrite(ofname, 255-img_ref*255)
        ref_path.append(ofname)
    

square
shape
this circle


In [14]:
reload(al)
reload(ar)
# get tracings from the data folder
col_names = ['session_id','age', 'tracing_item', 'pre_tran', 'post_tran', 'rotate', 'translate', 'scale']

for si, s in enumerate(m_id[400:]):
    # format cdm_run_v3(4)xxxxxxxxx
    child_info = s.split('_')
    version = child_info[2][1]
    
    for ti, t in enumerate(tracing):
        # check whether the tracing item already exists in the dataframe
        if ((data['tracing_item'] == t) & (data['session_id'] == s)).any():
            print "Tracing {} for child {} already exists".format(t, s)
            continue
        
        s_cat = t.replace('this ','')
        current_stim = scoll.find_one({'session_id':s, 'category':s_cat})
        if current_stim == None:
            print "Tracing {} for child {} does not exists".format(t, s)
            continue
            
        # find the directory where current image exists
        data_folder = "tracing_cdm_run_v{}/{}".format(version, t)
        c_age = current_stim['age']
        img_name = "age{}_{}_{}.png".format(c_age, s, t)  # e.g. age10_cdm_run_v31534270688827_shape.png
        img_draw = os.path.join(data_folder, img_name)

        if os.path.exists(fname):
            img_ref = ref_path[ti]
            output_path = os.path.join(trans_dir, '{}_{}_wrap.png'.format(s, t))
            init_loss, final_loss, rotate, translate, scale, warped = ar.affine_reg(img_draw, img_ref, output_path)
        
            new_stim = np.array([[s, c_age, t, init_loss, final_loss, rotate, translate, scale]])
            
            if si%20 == 0:
                print si, tracing[ti], init_loss, final_loss
                
            df2 = pd.DataFrame(new_stim, columns=col_names)
            data = data.append(df2, ignore_index=True)      
    
    if (si + 1)% 50 == 0:
        data.to_csv(data_output)
        

Tracing square for child cdm_run_v31526598908203 already exists
Tracing shape for child cdm_run_v31526598908203 already exists
0 this circle -0.24070237577 -0.370326757431
Tracing square for child cdm_run_v31533161571311 already exists
Tracing shape for child cdm_run_v31533161571311 already exists
Tracing square for child cdm_run_v31528063199159 already exists
Tracing shape for child cdm_run_v31528063199159 does not exists
Tracing this circle for child cdm_run_v31528063199159 does not exists
Tracing square for child cdm_run_v31528672799172 does not exists
Tracing shape for child cdm_run_v31528672799172 does not exists
Tracing square for child cdm_run_v31534453047819 does not exists
Tracing shape for child cdm_run_v31534453047819 does not exists
Tracing square for child cdm_run_v31530655612989 already exists
Tracing shape for child cdm_run_v31530655612989 already exists
Tracing square for child cdm_run_v31528483829261 does not exists
Tracing shape for child cdm_run_v31528483829261 does 

Tracing square for child cdm_run_v31530556898807 already exists
Tracing shape for child cdm_run_v31530556898807 does not exists
Tracing this circle for child cdm_run_v31530556898807 does not exists
Tracing square for child cdm_run_v31526598587970 does not exists
Tracing shape for child cdm_run_v31526598587970 does not exists
Tracing square for child cdm_run_v31528238200865 does not exists
Tracing shape for child cdm_run_v31528238200865 already exists
Tracing square for child cdm_run_v31528574186310 already exists
Tracing shape for child cdm_run_v31528574186310 already exists
Tracing square for child cdm_run_v31531421394850 already exists
Tracing shape for child cdm_run_v31531421394850 already exists
Tracing square for child cdm_run_v31526584027584 does not exists
Tracing shape for child cdm_run_v31526584027584 already exists
Tracing square for child cdm_run_v31526593619763 already exists
Tracing shape for child cdm_run_v31526593619763 already exists
Tracing square for child cdm_run_v31

Tracing square for child cdm_run_v31525992411710 does not exists
Tracing shape for child cdm_run_v31525992411710 does not exists
Tracing square for child cdm_run_v31528835999112 already exists
Tracing shape for child cdm_run_v31528835999112 already exists
Tracing square for child cdm_run_v31529703938903 already exists
Tracing shape for child cdm_run_v31529703938903 does not exists
Tracing square for child cdm_run_v31526763408835 already exists
Tracing shape for child cdm_run_v31526763408835 already exists
120 this circle -0.105864010751 -0.618656039238
Tracing square for child cdm_run_v31526506824890 already exists
Tracing shape for child cdm_run_v31526506824890 does not exists
Tracing square for child cdm_run_v31529182799206 does not exists
Tracing shape for child cdm_run_v31529182799206 already exists
Tracing this circle for child cdm_run_v31529182799206 does not exists
Tracing square for child cdm_run_v31528053791790 already exists
Tracing shape for child cdm_run_v31528053791790 alr

Tracing square for child cdm_run_v31528136999209 already exists
Tracing shape for child cdm_run_v31528136999209 already exists
Tracing square for child cdm_run_v31528481904629 already exists
Tracing shape for child cdm_run_v31528481904629 already exists
Tracing square for child cdm_run_v31532645032182 already exists
Tracing shape for child cdm_run_v31532645032182 does not exists
Tracing this circle for child cdm_run_v31532645032182 does not exists
Tracing square for child cdm_run_v31529426399365 already exists
Tracing shape for child cdm_run_v31529426399365 already exists
Tracing square for child cdm_run_v31528919459689 already exists
Tracing shape for child cdm_run_v31528919459689 already exists
Tracing square for child cdm_run_v31525893700992 already exists
Tracing shape for child cdm_run_v31525893700992 already exists
Tracing this circle for child cdm_run_v31525893700992 does not exists
Tracing square for child cdm_run_v31525893976840 already exists
Tracing shape for child cdm_run_v

Tracing square for child cdm_run_v31528321132428 does not exists
Tracing shape for child cdm_run_v31528321132428 already exists
Tracing this circle for child cdm_run_v31528321132428 does not exists
Tracing square for child cdm_run_v31533239510548 already exists
Tracing shape for child cdm_run_v31533239510548 does not exists
Tracing this circle for child cdm_run_v31533239510548 does not exists
Tracing square for child cdm_run_v31525899254114 does not exists
Tracing shape for child cdm_run_v31525899254114 already exists
Tracing square for child cdm_run_v31528835580022 does not exists
Tracing shape for child cdm_run_v31528835580022 already exists
Tracing this circle for child cdm_run_v31528835580022 does not exists
Tracing square for child cdm_run_v31529359799192 already exists
Tracing shape for child cdm_run_v31529359799192 already exists
Tracing square for child cdm_run_v31532722797926 does not exists
Tracing shape for child cdm_run_v31532722797926 already exists
Tracing this circle for

280 this circle -0.089794665575 -0.598332226276
Tracing square for child cdm_run_v31528224184593 does not exists
Tracing shape for child cdm_run_v31528224184593 already exists
Tracing square for child cdm_run_v31529092081104 does not exists
Tracing shape for child cdm_run_v31529092081104 already exists
Tracing square for child cdm_run_v31528845599021 does not exists
Tracing shape for child cdm_run_v31528845599021 already exists
Tracing square for child cdm_run_v31530742492388 already exists
Tracing shape for child cdm_run_v31530742492388 already exists
Tracing this circle for child cdm_run_v31530742492388 does not exists
Tracing square for child cdm_run_v31528399548613 already exists
Tracing shape for child cdm_run_v31528399548613 already exists
Tracing square for child cdm_run_v31532114536745 does not exists
Tracing shape for child cdm_run_v31532114536745 already exists
Tracing square for child cdm_run_v31531853118396 does not exists
Tracing shape for child cdm_run_v31531853118396 alr

Tracing this circle for child cdm_run_v31528395599167 does not exists
Tracing square for child cdm_run_v31531506560996 does not exists
Tracing this circle for child cdm_run_v31528406817418 does not exists
Tracing square for child cdm_run_v31525904067360 does not exists
Tracing this circle for child cdm_run_v31525904067360 does not exists
Tracing square for child cdm_run_v31528828735944 does not exists
Tracing this circle for child cdm_run_v31528828735944 does not exists
Tracing shape for child cdm_run_v31526168207854 does not exists
Tracing this circle for child cdm_run_v31526168207854 does not exists
Tracing square for child cdm_run_v31526510518453 does not exists
Tracing shape for child cdm_run_v31526510518453 does not exists
Tracing shape for child cdm_run_v31528137599244 does not exists
Tracing square for child cdm_run_v31526492679058 does not exists
Tracing square for child cdm_run_v31533506237294 does not exists
Tracing shape for child cdm_run_v31530644143472 does not exists
Trac

Tracing this circle for child cdm_run_v31529278944908 does not exists
Tracing square for child cdm_run_v31529178231083 does not exists
Tracing this circle for child cdm_run_v31529178231083 does not exists
Tracing square for child cdm_run_v31529351694802 does not exists
Tracing shape for child cdm_run_v31529351694802 does not exists
Tracing shape for child cdm_run_v31529003792315 does not exists
Tracing this circle for child cdm_run_v31529003792315 does not exists
Tracing this circle for child cdm_run_v31526422835385 does not exists
Tracing shape for child cdm_run_v31531605812435 does not exists
Tracing this circle for child cdm_run_v31531605812435 does not exists
Tracing shape for child cdm_run_v31529525671100 does not exists
Tracing this circle for child cdm_run_v31529525671100 does not exists
Tracing shape for child cdm_run_v31527886799373 does not exists
Tracing this circle for child cdm_run_v31527886799373 does not exists
Tracing square for child cdm_run_v31526580960319 does not ex

Tracing square for child cdm_run_v31531339169894 does not exists
Tracing square for child cdm_run_v31528828799227 does not exists
Tracing this circle for child cdm_run_v31528828799227 does not exists
Tracing shape for child cdm_run_v31528838259834 does not exists
Tracing this circle for child cdm_run_v31528838259834 does not exists
Tracing square for child cdm_run_v31528581653329 does not exists
Tracing shape for child cdm_run_v31528581653329 does not exists
Tracing shape for child cdm_run_v31525893144594 does not exists
Tracing this circle for child cdm_run_v31525893144594 does not exists
Tracing shape for child cdm_run_v31530399741780 does not exists
Tracing this circle for child cdm_run_v31530399741780 does not exists
Tracing shape for child cdm_run_v31528054199286 does not exists
Tracing this circle for child cdm_run_v31528054199286 does not exists
Tracing square for child cdm_run_v31529451599071 does not exists
Tracing shape for child cdm_run_v31529451599071 does not exists
560 sq

Tracing this circle for child cdm_run_v31528577999202 does not exists
620 square -0.640713989735 -0.839822590351
Tracing shape for child cdm_run_v31531517836477 does not exists
Tracing this circle for child cdm_run_v31531517836477 does not exists


In [15]:
data.to_csv(data_output)

In [ ]:
really_run_this = 1 
## compile data
if really_run_this:
    data = pd.DataFrame([sessionid, age, trace_item, pre_mse, post_mse, rotate, translate, scale])
    data = data.transpose()
    data.columns = ['session_id','age', 'tracing_item', 'pre_tran', 'post_tran', 'rotate', 'translate', 'scale']
    
## save out the data
data.to_csv(data_output)

#### Data Visualization

In [ ]:
really_run_this = 1 

if really_run_this:
    data = pd.read_csv(data_output)

- X axis: Age        
- Y axis: pre/post-mse

In [ ]:
#ylim = (0, 0.1)   # mse
ylim = (-0.8, 0.1)
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    age = []
    mse_list = []
    current_trace = data.loc[(data['tracing_item'] == t)]
    
    for index, row in current_trace.iterrows():
        age.append(row['age'])
        mse_list.append(row['pre_mse'])
    
    plt.subplot(1,3,ti+1)
    h = plt.scatter(age, mse_list)
    plt.ylabel(t +' pre-mse')
    plt.xlabel('age')
    plt.ylim(ylim)
    
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    age = []
    mse_list = []
    current_trace = data.loc[(data['tracing_item'] == t)]
    
    for index, row in current_trace.iterrows():
        age.append(row['age'])
        mse_list.append(row['post_mse'])
    
    plt.subplot(1,3,ti+1)
    h = plt.scatter(age, mse_list)
    plt.ylabel(t +' post-mse')
    plt.xlabel('age')
    plt.ylim(ylim)


- X axis: Pre-mse
- Y axis: Post-mse  
- Age: different colors

In [ ]:
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    plt.subplot(1,3,ti+1)
    plt.ylabel(t +' post-mse')
    plt.xlabel('pre-mse')
    plt.xlim((0.0, 0.1))
    plt.ylim((0.0, 0.1))
    
    for age in range(4, 8):
        current_trace = data.loc[(data['age'] == age) &(data['tracing_item'] == t)]
        pre_mse_list = []
        post_mse_list = []
        
        for index, row in current_trace.iterrows():
            pre_mse_list.append(row['pre_mse'])
            post_mse_list.append(row['post_mse'])

        h = plt.scatter(pre_mse_list, post_mse_list, label='age'+str(age))
    
    plt.legend()


- X axis: Age        
- Y axis: rotation

In [ ]:
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    age = []
    mse_list = []
    current_trace = data.loc[(data['tracing_item'] == t)]
    
    for index, row in current_trace.iterrows():
        age.append(row['age'])
        mse_list.append(row['rotate'])
    
    plt.subplot(1,3,ti+1)
    h = plt.scatter(age, mse_list)
    plt.ylabel(t +' rotation')
    plt.xlabel('age')
    plt.ylim((0.0, 0.2))

- X axis: Age        
- Y axis: translation

In [ ]:
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    age = []
    mse_list = []
    current_trace = data.loc[(data['tracing_item'] == t)]
    
    for index, row in current_trace.iterrows():
        age.append(row['age'])
        mse_list.append(row['translate'])
    
    plt.subplot(1,3,ti+1)
    h = plt.scatter(age, mse_list)
    plt.ylabel(t +' translation')
    plt.xlabel('age')
    plt.ylim((0.0, 0.25))

- X axis: Age
- Y axis: Scaling

In [ ]:
plt.figure(figsize=(18,5))
plt.subplots_adjust(wspace=0.3)

for ti, t in enumerate(tracing):
    age = []
    mse_list = []
    current_trace = data.loc[(data['tracing_item'] == t)]
    
    for index, row in current_trace.iterrows():
        age.append(row['age'])
        mse_list.append(row['scale'])
    
    plt.subplot(1,3,ti+1)
    h = plt.scatter(age, mse_list)
    plt.ylabel(t +' scale')
    plt.xlabel('age')
    plt.ylim((0, 1.2))

#### Tracing and Recognition

In [ ]:
# get recognition correction percentage
recog = pd.read_csv('photodraw_recognition_ratings.csv')
all_sessions = data.session_id.unique()

data['correct'] = 0
data['condition'] = 'None'

for index, id in enumerate(all_sessions):
    rec_col = recog.loc[(recog['sessionId'] == id)]
    if rec_col.shape[0] == 0:
        data.loc[data['session_id'] == id, ['correct']] = 0
        continue
    current_condition = rec_col['condition'].iloc[0]
    rec_correct_percent = rec_col.loc[(rec_col['correct'])].shape[0]/rec_col.shape[0]
    data.loc[data['session_id'] == id, ['correct']] = rec_correct_percent
    data.loc[data['session_id'] == id, ['condition']] = current_condition

data.to_csv("photodraw_tracing.csv")

In [ ]:


for ci, c in enumerate(['S','W','P']):
    plt.figure(figsize=(18,5))
    plt.subplots_adjust(wspace=0.3)
    for ti, t in enumerate(tracing):
        plt.subplot(1,3,ti+1)
        plt.ylabel(c + ' correct percent')
        plt.xlabel(t +' post-mse')
        plt.xlim(0.0,0.1)
        plt.ylim(0.0,1.0)
        
        for age in range(4, 8):
            current_trace = data.loc[(data['age'] == age) &(data['tracing_item'] == t) & (data['condition'] == c)]
            post_mse_list = []
            correct = []

            for index, row in current_trace.iterrows():
                post_mse_list.append(row['post_mse'])
                correct.append(row['correct'])

            h = plt.scatter(post_mse_list, correct, label='age'+str(age))
    
        plt.legend()



#### Evaluate whether the overall metric captures human judgment intuitions

In [ ]:
# Overlap the drawing on reference images
current_data_dir = data_dir = os.path.join(analysis_dir,"museumstation_v4_tracing")
tracing = ['square', 'shape', 'this circle']

overlap_dir = os.path.join(analysis_dir, 'overlap')
# get tracings from the data folder
for child_dir in os.listdir(current_data_dir):
    if child_dir.startswith('cdm'):
        child_info = child_dir.split('_')
        #c_age = int(child_info[-1].replace('age',''))
        c_age = child_info[-1]
        c_sessionid = child_info[2]

        for ti, t in enumerate(tracing):
            img_draw = os.path.join(data_dir, child_dir, '{}.png'.format(t))
            
            if os.path.exists(img_draw):
                img_ref = ref_path[ti]
                if t == 'this square':
                    t = 'square'
                if t == 'this shape':
                    t = 'shape'
                output_path = '{}/{}_{}.png'.format(t, c_sessionid, c_age)
                
                draw = cv2.imread(img_draw).astype(int16)
                ref = cv2.imread(img_ref).astype(int16)
                ref[ref<255] = 150
                draw[np.sum(draw<255,axis=2)] = [255,0,0]
                img = np.add(draw, ref) /2
                
                cv2.imwrite(os.path.join(overlap_dir,output_path), img)
                

In [ ]:
# Overlap the drawing on reference images
current_data_dir = os.path.join(analysis_dir,"tracing_cdm_run_v3")
tracing = ['square', 'shape', 'this circle']

overlap_dir = os.path.join(analysis_dir, 'overlap_v3')
if not os.path.exists(overlap_dir):
    os.makedirs(overlap_dir)

# get tracings from the data folder
for ti, t in enumerate(tracing):
    overlap_t = os.path.join(overlap_dir, t)
    if not os.path.exists(overlap_t):
        os.makedirs(overlap_t)
        
    t_dir = os.path.join(current_data_dir, t)
        
    for fname in os.listdir(t_dir):
        if fname.startswith('age'):
            img_draw = os.path.join(current_data_dir, t_dir, fname)
            img_ref = ref_path[ti]

            draw = cv2.imread(img_draw).astype(int16)
            ref = cv2.imread(img_ref).astype(int16)
            img = ref
            img[np.sum(ref, axis=2)<255*3] = [150, 150, 150]
            img[np.sum(draw, axis=2)<255*3] = [40, 40, 170]
            

            cv2.imwrite(os.path.join(overlap_dir,t, fname), img)


In [ ]:
really_run_this = 1 
data_output = 'photodraw_tracing_ncc.csv'
if really_run_this:
    data = pd.read_csv(data_output)

In [ ]:
# convert three spatial errors into one single number
# 1. normalize rotation, scaling, and translation
data['norm_r'] = (data['rotate'] - data['rotate'].mean())/data['rotate'].std()
data['norm_t'] = (data['translate'] - data['translate'].mean())/data['translate'].std()
data['norm_s'] = (data['scale'] - data['scale'].mean())/data['scale'].std()

# 2. summarize the three varaibles
w_r, w_t, w_s = 1, 1, 1
data['spatial'] = data['norm_r'] * w_r + data['norm_t'] * w_t + data['norm_s'] * w_s
data.to_csv(data_output)


In [ ]:
rating = pd.read_csv('photodraw_tracing_rating.csv')
spatial_error = data['spatial']
shape_error = data['post_tran']
rater1 = rating['rater1']

norm_spatial = (spatial_error - np.mean(spatial_error))/np.std(spatial_error)
norm_shape = (shape_error - np.mean(shape_error))/np.std(shape_error)
overall_error = norm_spatial + norm_shape

spatial_cor = pearsonr(spatial_error, rater1)
shape_cor = pearsonr(shape_error, rater1)
overall_cor = pearsonr(overall_error, rater1)

print 'NCC loss'
print 'correlate human judgments with spatial errors: r = {}, p = {}'.format(spatial_cor[0], spatial_cor[1])
print 'correlate human judgments with shape errors: r = {}, p = {}'.format(shape_cor[0], shape_cor[1]) 
print 'correlate human judgments with shape+spatial errors: r = {}, p = {}'.format(overall_cor[0], overall_cor[1])


In [ ]:
data['rater1'] = rater1
data.to_csv('photodraw_summary_{}.csv'.format(loss))

#### Compare different loss

In [ ]:
xlim = (-1,0)
ylim = (0, 0.08)

ncc = pd.read_csv('photodraw_tracing_ncc.csv')
mse = pd.read_csv('photodraw_tracing_mse.csv')

ncc_shape, ncc_spatial, ncc_overall = [], [], []
mse_shape, mse_spatial, mse_overall = [], [], []

for s in all_sessions:
    current_ncc = ncc.loc[(ncc['session_id'] == s)]
    
    for index, draw in current_ncc.iterrows():
        ncc_shape.append(draw['post_tran'])
        ncc_spatial.append(draw['spatial'])
        ncc_overall.append(draw['post_tran'] + draw['spatial'])
        
        current_mse = mse.loc[(mse['session_id'] == s) & (mse['tracing_item'] == draw['tracing_item'])]
        mse_shape.append(current_mse['post_tran'].item())
        mse_spatial.append(current_mse['spatial'].item())
        mse_overall.append(current_mse['post_tran'].item() + current_mse['spatial'].item())


plt.figure(figsize=(16,5))
plt.subplot(1,2,1)
h = plt.scatter(ncc_shape, mse_shape)
plt.ylabel('MSE Shape Error')
plt.xlabel('NCC Shape Error')

plt.subplot(1,2,2)
h = plt.scatter(ncc_spatial, mse_spatial)
plt.ylabel('MSE Spatial Error')
plt.xlabel('NCC Spatial Error')


In [ ]:
shape_loss_cor = pearsonr(ncc_shape, mse_shape)
spatial_loss_cor = pearsonr(ncc_spatial, mse_spatial)
overall_loss_cor = pearsonr(ncc_overall, mse_overall)

print "NCC loss vs. MSE loss Pearson'r Correlation"
print 'correlate NCC shape error with MSE shape errors: r = {}, p = {}'.format(shape_loss_cor[0], shape_loss_cor[1])
print 'correlate NCC spatial error with MSE spatial errors: r = {}, p = {}'.format(spatial_loss_cor[0], spatial_loss_cor[1])
print 'correlate NCC overall error with MSE overall errors: r = {}, p = {}'.format(overall_loss_cor[0], overall_loss_cor[1])

